# Example Experiment
> Experiment using Repeated MNIST and BatchBALD vs BALD vs random sampling

This notebook ties everything together and runs an AL loop.

In [1]:
# experiment

#import blackhc.project.script
from tqdm.auto import tqdm

In [2]:
# experiment

import math
import time
import torch
from torch import nn as nn
from torch.nn import functional as F

from batchbald_redux import (
    active_learning,
    batchbald,
    consistent_mc_dropout,
    joint_entropy,
    repeated_mnist,
)
torch.manual_seed(0)


Let's define our Bayesian CNN model that we will use to train MNIST.

In [3]:
# experiment


class BayesianCNN(consistent_mc_dropout.BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = consistent_mc_dropout.ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input

In [4]:
from torchvision.datasets import MNIST,FashionMNIST,CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb

Grab our dataset, we'll use Repeated-MNIST. We will acquire to samples for each class for our initial training set.

In [ ]:
# experiment

for set_pool_size in [1000,3000,5000,10000,20000,40000]:

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    train_dataset = MNIST("data", train=True, transform=transform)
    test_dataset = MNIST("data", train=False,transform=transform)

    train_dataset,monitor_set = random_split(
        train_dataset, [59900,100]
    )
    num_initial_samples = 20
    num_classes = 10

    initial_samples = active_learning.get_balanced_sample_indices(
        repeated_mnist.get_targets(train_dataset), num_classes=num_classes, n_per_digit=num_initial_samples / num_classes
    )

    # experiment
    max_training_samples = 300
    acquisition_batch_size = 5
    num_inference_samples = 100
    num_test_inference_samples = 5
    num_samples = 100000

    test_batch_size = 512
    batch_size = 64
    scoring_batch_size = 128
    training_iterations = 4096 
    use_cuda = torch.cuda.is_available()

    print(f"use_cuda: {use_cuda}")

    device = "cuda" if use_cuda else "cpu"

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

    active_learning_data = active_learning.ActiveLearningData(train_dataset)
    active_learning_data.acquire(initial_samples)

    # THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
    active_learning_data.extract_dataset_from_pool(59900-set_pool_size)

    train_loader = torch.utils.data.DataLoader(
        active_learning_data.training_dataset,
        sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, training_iterations),
        batch_size=batch_size,
        **kwargs,
        )
    print(len(active_learning_data.training_dataset))
    
    pool_loader = torch.utils.data.DataLoader(
        active_learning_data.pool_dataset, batch_size=scoring_batch_size, shuffle=False, **kwargs
    )
    
  
    wandb.init(project="ActiveLearning_1127_MNIST")
    wandb.run.name = f'BALD_{set_pool_size}'
    wandb.run.save()


    # Run experiment
    test_accs = []
    test_loss = []
    added_indices = []

    pbar = tqdm(initial=len(active_learning_data.training_dataset), total=max_training_samples, desc="Training Set Size")
    epoch=0



    while True:
        model = BayesianCNN(num_classes).to(device=device)
        optimizer = torch.optim.Adam(model.parameters())
      
        epoch+=1
        model.train()
        train_loss=0
        # Train
        for data, target in tqdm(train_loader, desc="Training", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            optimizer.zero_grad()

            prediction = model(data, 1).squeeze(1)
            loss = F.nll_loss(prediction, target)
            train_loss+=loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        # Test
        loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc="Testing", leave=False):
                data = data.to(device=device)
                target = target.to(device=device)

                prediction = torch.logsumexp(model(data, num_test_inference_samples), dim=1) - math.log(
                    num_test_inference_samples
                )
                loss += F.nll_loss(prediction, target, reduction="sum")

                prediction = prediction.max(1)[1]
                correct += prediction.eq(target.view_as(prediction)).sum().item()

        loss /= len(test_loader.dataset)

        percentage_correct = 100.0 * correct / len(test_loader.dataset)

        if len(active_learning_data.training_dataset) >= max_training_samples:
            break

        # Acquire pool predictions

        acquir_start=time.time()
        N = len(active_learning_data.pool_dataset)
        logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)

        with torch.no_grad():
            model.eval()

            for i, (data, _) in enumerate(tqdm(pool_loader, desc="Evaluating Acquisition Set", leave=False)):
                data = data.to(device=device)

                lower = i * pool_loader.batch_size
                upper = min(lower + pool_loader.batch_size, N)
                logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)


        with torch.no_grad():
#                candidate_batch = batchbald.get_batchbald_batch(
#                    logits_N_K_C, acquisition_batch_size, num_samples, dtype=torch.double, device=device
#                )
             candidate_batch = batchbald.get_bald_batch(
                 logits_N_K_C, acquisition_batch_size, dtype=torch.double, device=device
             )

        acquire_end=acquir_start-time.time()
        wandb.log({"Epoch":epoch,
                   "Num_Pool": len(active_learning_data.pool_dataset),
                   "Num_Train":len(active_learning_data.training_dataset),
                   "Test Loss": loss,
                   'Train Loss': train_loss,
                  "Test Accuracy":percentage_correct,
                   "Query Time":-acquire_end})


        targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
        dataset_indices = active_learning_data.get_dataset_indices(candidate_batch.indices)
#
        print("Dataset indices: ", dataset_indices)
        print("Scores: ", candidate_batch.scores)
        print("Labels: ", targets[candidate_batch.indices])

        active_learning_data.acquire(candidate_batch.indices)
        added_indices.append(dataset_indices)
        pbar.update(len(dataset_indices))

use_cuda: True
20


wandb: Currently logged in as: hslrock (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [57107 55423 26466 36529 31497]
Scores:  [0.9414304363415504, 0.9044963264475723, 0.8603897725485006, 0.8447358442541467, 0.8340402439270416]
Labels:  tensor([8, 8, 8, 2, 3])


Dataset indices:  [58948 26864 24341 23779 57557]
Scores:  [0.9854901483649576, 0.964919208507462, 0.9433214168168464, 0.9319478544947193, 0.9092470223903416]
Labels:  tensor([0, 1, 8, 9, 7])


...

Dataset indices:  [ 5417 52400 44669 52858 12652]
Scores:  [0.523827942953021, 0.4892486719412428, 0.4859476149305504, 0.4852484755895199, 0.4756494002589511]
Labels:  tensor([4, 4, 0, 9, 4])


Dataset indices:  [ 2385 54400  2388  2360 12693]
Scores:  [0.5040658365282513, 0.5020820414835003, 0.4964660077414602, 0.4958105819611638, 0.484082081577047]
Labels:  tensor([9, 3, 8, 3, 2])


Dataset indices:  [10442 59225 10455 11277 18399]
Scores:  [0.5819211355387293, 0.5807654856990665, 0.5790855366490519, 0.5786813749513269, 0.573529666074629]
Labels:  tensor([5, 7, 4, 4, 7])


Dataset indices:  [ 1171 49387 54813 41099 20136]
Scores:  [0.46919038194065554, 0.46838458686155804, 0.4675914382156806, 0.464827319362914, 0.46233431252378376]
Labels:  tensor([6, 6, 6, 9, 4])


Dataset indices:  [ 6938 45449 57908 38184 27275]
Scores:  [0.44447574186013883, 0.43754610435408603, 0.43031910206041757, 0.4270554686231862, 0.42657507035354203]
Labels:  tensor([9, 0, 0, 2, 7])


For this example, we are going to take two shortcuts that will reduce the performance:
* we discard most of the training set and only keep 20k samples; and
* we don't implement early stopping or epoch-wise training.

Instead, we always train by drawing 24576 many samples from the training set. This will overfit in the beginning and underfit later, but it still is sufficient to achieve 90% accuracy with 105 samples in the training set. 

In [8]:
# experiment
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

for set_pool_size in [500,1000]:

    train_dataset = FashionMNIST("data", train=True, download=True,transform=transform)
    test_dataset = FashionMNIST("data", train=False, download=True,transform=transform)

    train_dataset,monitor_set = random_split(
        train_dataset, [59900,100]
    )
    num_initial_samples = 20
    num_classes = 10

    initial_samples = active_learning.get_balanced_sample_indices(
        repeated_mnist.get_targets(train_dataset), num_classes=num_classes, n_per_digit=num_initial_samples / num_classes
    )

    # experiment
    max_training_samples = 300
    acquisition_batch_size = 5
    num_inference_samples = 100
    num_test_inference_samples = 5
    num_samples = 100000

    test_batch_size = 512
    batch_size = 64
    scoring_batch_size = 128
    training_iterations = 4096 * 6
    use_cuda = torch.cuda.is_available()

    print(f"use_cuda: {use_cuda}")

    device = "cuda" if use_cuda else "cpu"

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

    active_learning_data = active_learning.ActiveLearningData(train_dataset)

    # Split off the initial samples first.
    active_learning_data.acquire(initial_samples)

    # THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
    active_learning_data.extract_dataset_from_pool(59900-set_pool_size)

    train_loader = torch.utils.data.DataLoader(
        active_learning_data.training_dataset,
        sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, training_iterations),
        batch_size=batch_size,
        **kwargs,
    )

    pool_loader = torch.utils.data.DataLoader(
        active_learning_data.pool_dataset, batch_size=scoring_batch_size, shuffle=False, **kwargs
    )
    wandb.init(project="ActiveLearning_1124_FASHION")
    wandb.run.name = f'BALD_{set_pool_size}'
    wandb.run.save()


    # Run experiment
    test_accs = []
    test_loss = []
    added_indices = []

    pbar = tqdm(initial=len(active_learning_data.training_dataset), total=max_training_samples, desc="Training Set Size")
    epoch=0
    model = BayesianCNN(num_classes).to(device=device)
    optimizer = torch.optim.Adam(model.parameters())

    while True:

        epoch+=1
        model.train()
        train_loss=0
        # Train
        for data, target in tqdm(train_loader, desc="Training", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            optimizer.zero_grad()

            prediction = model(data, 1).squeeze(1)
            loss = F.nll_loss(prediction, target)
            train_loss+=loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        # Test
        loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc="Testing", leave=False):
                data = data.to(device=device)
                target = target.to(device=device)

                prediction = torch.logsumexp(model(data, num_test_inference_samples), dim=1) - math.log(
                    num_test_inference_samples
                )
                loss += F.nll_loss(prediction, target, reduction="sum")

                prediction = prediction.max(1)[1]
                correct += prediction.eq(target.view_as(prediction)).sum().item()

        loss /= len(test_loader.dataset)

        percentage_correct = 100.0 * correct / len(test_loader.dataset)



        if len(active_learning_data.training_dataset) >= max_training_samples:
            break

        # Acquire pool predictions

        acquir_start=time.time()
        N = len(active_learning_data.pool_dataset)
        logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)

        with torch.no_grad():
            model.eval()

            for i, (data, _) in enumerate(tqdm(pool_loader, desc="Evaluating Acquisition Set", leave=False)):
                data = data.to(device=device)

                lower = i * pool_loader.batch_size
                upper = min(lower + pool_loader.batch_size, N)
                logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)


        with torch.no_grad():
#                candidate_batch = batchbald.get_batchbald_batch(
#                    logits_N_K_C, acquisition_batch_size, num_samples, dtype=torch.double, device=device
#                )
             candidate_batch = batchbald.get_bald_batch(
                 logits_N_K_C, acquisition_batch_size, dtype=torch.double, device=device
             )

        acquire_end=acquir_start-time.time()
        wandb.log({"Epoch":epoch,
                   "Num_Pool": len(active_learning_data.pool_dataset),
                   "Num_Train":len(active_learning_data.training_dataset),
                   "Test Loss": loss,
                   'Train Loss': train_loss,
                  "Test Accuracy":percentage_correct,
                   "Query Time":-acquire_end})


        targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
        dataset_indices = active_learning_data.get_dataset_indices(candidate_batch.indices)

        print("Dataset indices: ", dataset_indices)
        print("Scores: ", candidate_batch.scores)
        print("Labels: ", targets[candidate_batch.indices])

        active_learning_data.acquire(candidate_batch.indices)
        added_indices.append(dataset_indices)
        pbar.update(len(dataset_indices))

use_cuda: True


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Num_Pool,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
Num_Train,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Query Time,▂▅▄▃▃▄▃▁▂▆▄▂▄▃▄▃▂▄█▄▁▆▂▅▆▃▅▃▄▅▃▅▅▄▃▆▅▄▂▃
Test Accuracy,▁▂▂▃▃▅▄▅▆▅▆▆▆▆▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████
Test Loss,▆█▇▇▆▆▇▆▆▇█▆▅▅▆▆▆▅▅▅▅▃▃▃▃▂▃▃▃▂▃▂▂▃▂▂▁▂▂▁
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▂▂▁▂▂▁▂▁▂▂▂▂▂▂
Epoch,56
Num_Pool,705
Num_Train,295
Query Time,4.96489


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [59541 59247 46722 29755 44012]
Scores:  [1.1034674009089207, 0.9731317950650874, 0.9579382660065318, 0.9378319640951753, 0.8973348390108122]
Labels:  tensor([1, 1, 3, 2, 2])


...

Dataset indices:  [55974 50107 39369  4233 50332]
Scores:  [0.8618953788083814, 0.7538565284006618, 0.7331481935829736, 0.732965920962983, 0.6955242562665395]
Labels:  tensor([8, 9, 4, 2, 4])


Dataset indices:  [43033 45132  2597 49042  7937]
Scores:  [0.7094518192888651, 0.6892724303582638, 0.6717764329842295, 0.6705559192758017, 0.6663905234610867]
Labels:  tensor([9, 4, 2, 4, 6])


Dataset indices:  [48670  3078 22971 26233 10755]
Scores:  [0.796750023753616, 0.7837503907577841, 0.6895045319113582, 0.6623850084906869, 0.6585760940670757]
Labels:  tensor([2, 9, 9, 4, 3])


Dataset indices:  [12450 13788 32086 42232 33312]
Scores:  [0.7170936338747449, 0.6876835726326923, 0.6859178931133103, 0.6836032670342456, 0.6831195850136548]
Labels:  tensor([7, 1, 9, 0, 2])


Dataset indices:  [57737   199 45627 38163 15289]
Scores:  [0.7864369435355899, 0.7466614008470877, 0.6844646052775212, 0.6585582405311193, 0.6531372946974765]
Labels:  tensor([3, 3, 6, 2, 7])


use_cuda: True


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Num_Pool,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
Num_Train,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Query Time,▆▆▆▆▇▆▆▆▆▆▆▆▂▂▂▂██▁▁▁▅▁▆▆▁▆▆▆▆▆▆▇▆▆▇▂▇▂▇
Test Accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▆▆▆▆▆▇▇▇▇▇▇▇▇█▇█▇█▇███▇█████
Test Loss,█▇▆▆▅▅▅▅▄▄▄▄▄▅▄▅▄▄▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
Train Loss,█▁▁▁▂▁▁▁▁▁▁▂▁▁▂▁▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Epoch,56
Num_Pool,39705
Num_Train,295
Query Time,121.37527


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [12697  6003 25062 38471 53288]
Scores:  [1.2252151897454726, 1.1875483104792552, 1.126461825707276, 1.125566788269468, 1.1253783356421825]
Labels:  tensor([4, 8, 5, 5, 2])


...

Dataset indices:  [23316 10159 55678 10347 15898]
Scores:  [1.2283279138255505, 1.2088654615575578, 1.201935045605035, 1.1955714043718393, 1.1624459534814964]
Labels:  tensor([9, 6, 6, 9, 8])


Dataset indices:  [  215 32595 11418  8387   473]
Scores:  [1.2703730625710792, 1.210776742769179, 1.209314487157622, 1.1932151696951196, 1.185086428342859]
Labels:  tensor([1, 8, 6, 2, 0])


Dataset indices:  [55201 35748 44287 10676 44318]
Scores:  [1.2530522666550798, 1.1847800440956937, 1.1779101637125289, 1.1712733803044366, 1.1608593743824205]
Labels:  tensor([8, 0, 4, 4, 1])


Dataset indices:  [13293  5813 39797 13607 57035]
Scores:  [1.2897209540647236, 1.223301456619027, 1.2156255291055296, 1.2122844989724488, 1.203658117697146]
Labels:  tensor([5, 6, 2, 6, 6])


Dataset indices:  [56060 33085 31814 20566 58457]
Scores:  [1.303295335753478, 1.2219012849489583, 1.2049271190899074, 1.182632186168095, 1.1796006808495387]
Labels:  tensor([8, 8, 0, 8, 0])
